In [4]:
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import urllib.request
from konlpy.tag import Okt

import json

tokenizer = Tokenizer()

# wordInext 목록 받아오기
with open('LSTM_module_ver2.0_wordIndex.json') as f:
    word_index = json.load(f)
    tokenizer.word_index = word_index
    
with open('LSTM_module_ver2.0_variable.json') as f:
    var = json.load(f)
    
max_data = var['max_data'] # 데이터 문자열 최대 길이
min_data = var['min_data'] # 데이터 문자열 최소 길이
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)

model = tf.keras.models.load_model('LSTM_module_ver2.0.h5')
model.summary()

okt = Okt()

def preTreatment(data):
    stopwords = ['\n','.','?',',','']
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp


def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence]) # 단어를 숫자값, 인덱스로 변환하여 저장
    pad_new = pad_sequences(data, maxlen = max_len) # 모든 메일의 길이를 100로 설정 (빈 부분은 0으로 패딩)
    score = float(model.predict(pad_new))
    print("[{:.2f}% 확률로 스팸입니다.]".format(score * 100))
    print("-------------------------------------------------------------------")
    return score
        

# 음성파일 STT 변환
import speech_recognition as sr

r = sr.Recognizer()
harvard = sr.AudioFile('ham2.flac') # 100MB 용량 제한
with harvard as source:
    audio = r.record(source)

t = r.recognize_google(audio, language='ko-KR')

temp_X = []
x = [t[i:i+max_data] for i in range(0, len(t), max_data)] # 텍스트 파일 중 150 길이로 데이터 길이 제한

for s in x:
    if len(s) > min_data: # 길이가 50 이하인 데이터 삭제
        temp_X.append(s)
x = temp_X
score_sum = 0.0
for txt in x:
    print(txt)
    data = preTreatment(txt)
    score_sum += sentiment_predict(data)
score_result = score_sum / len(x)
print("{:.2f}% 확률로 스팸입니다.\n".format(score_result * 100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2466000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,583,377
Trainable params: 2,583,377
Non-trainable params: 0
_________________________________________________________________
여보세요 여보세요네 혹시 그 당근마켓 그 신발 판매 안 하시는 일 보고 전화 드렸는데요 나이키 나이키 조던 그거 15만원에 올리셨던데 그 혹시 좀 할인 안 될까요 제가 아직 학생이라서 돈이 많이 없는데 된다고 적어 놨는데 내 돼요 얼마까지 가능하신가요 얼마까지 마세요 
[89.92% 확률로 스팸입니다.]
-------------------------------------------------------------------
마음 같아서는 15만원까지 15만 원 15만 원 할인 15만 원 어디 사시죠 혹시 저는 거주중입니다 그러면 15만 원에 내일 하시죠 직거래 하시는 건가요 하면 될 거 같아요 근데 지금 요새 코로나 가지고 만나기가 좀 걸어 지는데 혹시 계좌 번호 